In [ ]:
from sympy import *
import numpy as np
import math 
import random

# Function Defining
ใช้ Gradient Descent ในการ Optimize


In [ ]:
def DeclareVariable(N): #P as price
    string = ""
    for i in range(N):
        if i !=N:
            string += "P"+str(i+1)+" "
        else:
            string += str(i+1)
    a = symbols(string, real=True)
    return a

def GetPartialDerivative(function,variable):
    f = function
    a = variable
    partial_derivative = [diff(f,i) for i in a]
    lambded_partial_derivative = [lambdify([x for x in a],function)\
                                  for i,function in enumerate(partial_derivative)]
    return partial_derivative, lambded_partial_derivative

def GradientAscent(lambded_f,lambded_partial_derivative,THRESHOLD,EPOCH_LIMIT,lr\
                    ,variable_count,random_range=None,coordinate=None): #random range get in list
    if not(random_range is None):
        coordinate=[float(random.randint(*random_range)) for i in range(variable_count)]
    elif coordinate is None:
        raise "Please input initial coordinate"
    threshold_check = THRESHOLD + 1
    epoch = 1
    while threshold_check>THRESHOLD and epoch<EPOCH_LIMIT :
        coordinate = [coordinate[i] + lr * function(*coordinate) \
                          for i,function in enumerate(lambded_partial_derivative)]
        threshold_check = max([lr * function(*coordinate) for function in lambded_partial_derivative])
        epoch+=1
    result = lambded_f(*coordinate)
    return result,coordinate

def GradientAscentMomentum(lambded_f,lambded_partial_derivative,THRESHOLD,EPOCH_LIMIT,lr,alpha\
                    ,variable_count,random_range=None,coordinate=None): #random range get in list
    if not(random_range is None):
        coordinate=[float(random.randint(*random_range)) for i in range(variable_count)]
    elif coordinate is None:
        raise "Please input initial coordinate"
    threshold_check = THRESHOLD + 1
    epoch = 1
    latest_dw = [0 for i in range(len(lambded_partial_derivative))]
    while threshold_check>THRESHOLD and epoch<EPOCH_LIMIT :
        coordinate = [coordinate[i] + lr * function(*coordinate) + alpha * latest_dw[i] \
                          for i,function in enumerate(lambded_partial_derivative)]
        threshold_check = max([lr * function(*coordinate)+ alpha * latest_dw[i]\
                               for i,function in enumerate(lambded_partial_derivative)])
        latest_dw = [lr * function(*coordinate) \
                          for i,function in enumerate(lambded_partial_derivative)] 
        epoch+=1
    result = lambded_f(*coordinate)
    return result,coordinate

class GradientDescent:
    def __init__(self,N,function,v):
        self.N = N
        self.v = v
        self.function = function
        self.lambded_f = lambdify([x for x in self.v],function)
        _,self.lambded_partial_derivative = GetPartialDerivative(function,self.v)
    def GradientAscentMomentum(self,Threshold,limit,lr,momentum,random_range = None,coordinate=None):
        return GradientAscentMomentum(self.lambded_f,self.lambded_partial_derivative,Threshold,limit,lr,momentum\
                                    ,self.N,random_range = random_range,coordinate = coordinate)
    def GradientAscent(self,Threshold,limit,lr,random_range = None,coordinate=None):
        return GradientAscent(self.lambded_f,self.lambded_partial_derivative,Threshold,limit,lr\
                                    ,self.N,random_range = random_range,coordinate = coordinate)

class PriceOPT:
    def __init__(self,Qd,Vc,Fc,P):
        self.Qd = Qd
        self.Vc = Vc
        self.Fc = Fc
        self.P = P
    
    def FindBestPrice(self,Threshold,limit,lr,momentum,random_range = None,coordinate=None):
        Qd_array = np.array(self.Qd)
        Vc_array = np.array(self.Vc)
        Fc_array = np.array(self.Fc)
        P = self.P 
        lambded_Qd_array = [lambdify([x for x in P],a) for i,a in enumerate(list(Qd_array))]
        QdP_array = np.array([a*P[i] for i,a in enumerate(list(Qd_array))])
        QdVc_array = np.array([a*list(Vc_array)[i] for i,a in enumerate(list(Qd_array))])

        Pi_array = QdP_array-QdVc_array-Fc_array
        Pi_list = list(Pi_array)
        total_Pi = np.sum(Pi_array)
        lambded_total_pi = lambdify([x for x in P],total_Pi)
        
        
        obj = GradientDescent(len(P),total_Pi,self.P)
        Profit,Price = obj.GradientAscentMomentum(Threshold,limit,lr,momentum,random_range)
        Sell = np.array([x(*[j for j in Price ]) for i,x in enumerate(list(lambded_Qd_array))])
        Cost = np.array([math.ceil(x) for x in Sell])*Vc_array + Fc_array
        
        return Profit,Price,list(Sell),list(Cost),total_Pi

# Example 1

In [ ]:
P = DeclareVariable(2)

Qd0 = 1500 - (P[0])
Qd1 = 100 - (P[1]/2) 
Qd = [Qd0,Qd1]
Vc = [1200,100]
Fc = [2600,600]

Threshold = 10**(-10)
limit = 100000
lr = 0.4
momentum = 0.3
random_range =[10,100]

model = PriceOPT(Qd,Vc,Fc,P)
Profit, Price, Sales, Cost, total_pi  = model.FindBestPrice(Threshold,limit,lr,momentum\
                                                           ,random_range = random_range)

In [ ]:
print(f"Profit is {round(Profit,1)}")
print(f"Price of Each Products is {[round(x,1) for x in Price]}")
print(f"Sales of Each Profuct is {[math.ceil(x) for x in Sales]}")
print(f"Cost of Each Product is {[round(x) for x in Cost]}")

Profit is 20550.0
Price of Each Products is [1350.0, 150.0]
Sales of Each Profuct is [150, 26]
Cost of Each Product is [182600, 3200]


# Example 2

In [ ]:
P = DeclareVariable(3)

Qd0 = 1500 - (P[0]/2) 
Qd1 = 10000 - (P[1]/2)
Qd2 = 1200 - (P[2]/3)
Qd = [Qd0,Qd1,Qd2]
Vc = [1200,120,100] #Variable Cost 
Fc = [2600,3200,1200] #Fixed-Cost

Threshold = 10**(-10)
limit = 100000 
lr = 0.4
momentum = 0.3
random_range =[10,100]

model = PriceOPT(Qd,Vc,Fc,P)
Profit, Price, Sales, Cost, total_pi = model.FindBestPrice(Threshold,limit,lr,momentum,random_range = random_range)

In [ ]:
print(f"Profit is {round(Profit,1)}")
print(f"Price of Each Products is {[round(x,1) for x in Price]}")
print(f"Sales of Each Profuct is {[math.ceil(x) for x in Sales]}")
print(f"Cost of Each Product is {[round(x) for x in Cost]}")

Profit is 50820633.3
Price of Each Products is [2100.0, 10060.0, 1850.0]
Sales of Each Profuct is [451, 4971, 584]
Cost of Each Product is [543800, 599720, 59600]
